In [ ]:
# src/cnn_classification.py
"""
cnn_classification.py
Fichier complet pour le TP3 : préparation des données CIFAR-10, définition d'un CNN classique,
définition d'un petit ResNet (3 blocs résiduels), entraînement et évaluation.

Usage (exécution en script):
    python src/cnn_classification.py

Ou importer les fonctions dans un notebook:
    from src.cnn_classification import build_basic_cnn, build_resnet_small, prepare_data
"""

import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

# -------------------------
# Configuration
# -------------------------
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 10
VALIDATION_SPLIT = 0.1
CHECKPOINT_DIR = "results/checkpoints"

# -------------------------
# Data preparation
# -------------------------
def prepare_data():
    # 1. Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    # Input shape
    INPUT_SHAPE = x_train.shape[1:]  # (32, 32, 3)

    # 2. Normalize pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # 3. Convert labels to one-hot
    y_train = keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

    print(f"Input data shape: {INPUT_SHAPE}")
    print(f"x_train shape: {x_train.shape}")
    print(f"y_train shape (after one-hot): {y_train.shape}")
    print(f"x_test shape: {x_test.shape}")
    print(f"y_test shape (after one-hot): {y_test.shape}")

    return (x_train, y_train), (x_test, y_test), INPUT_SHAPE

# -------------------------
# Basic CNN model
# -------------------------
def build_basic_cnn(input_shape, num_classes):
    model = keras.Sequential([
        # Conv Layer 1
        keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # Conv Layer 2
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # Flatten + Dense
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# -------------------------
# Residual block & small ResNet
# -------------------------
def residual_block(x, filters, kernel_size=(3, 3), stride=1):
    """
    Simplified residual block:
      - main path: conv -> relu -> conv
      - skip path: identity or 1x1 conv if stride>1 or channel mismatch
    """
    y = keras.layers.Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    y = keras.layers.Conv2D(filters, kernel_size, padding='same')(y)

    # Adjust skip connection when needed
    x_skip = x
    # If input channels != filters or stride > 1, adjust via 1x1 conv
    input_channels = int(x.shape[-1])
    if stride > 1 or input_channels != filters:
        x_skip = keras.layers.Conv2D(filters, (1, 1), strides=stride, padding='same')(x_skip)

    z = keras.layers.Add()([x_skip, y])
    z = keras.layers.Activation('relu')(z)
    return z

def build_resnet_small(input_shape, num_classes):
    """
    Build a small ResNet-like model using 3 residual blocks (as requested in the TP).
    Block configurations:
      - block1: filters=32, stride=1
      - block2: filters=64, stride=2 (reduces spatial dim)
      - block3: filters=64, stride=1
    """
    inputs = keras.Input(shape=input_shape)
    x = keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)

    x = residual_block(x, 32, stride=1)
    x = residual_block(x, 64, stride=2)
    x = residual_block(x, 64, stride=1)

    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='resnet_small')
    return model

# -------------------------
# Training and evaluation helpers
# -------------------------
def compile_and_train(model, x_train, y_train, x_val=None, y_val=None,
                      epochs=EPOCHS, batch_size=BATCH_SIZE, checkpoint_dir=CHECKPOINT_DIR):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    callbacks = []
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    ckpt_path = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}_valacc_{val_accuracy:.4f}.h5")
    callbacks.append(keras.callbacks.ModelCheckpoint(ckpt_path, save_best_only=False, save_weights_only=False))

    history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=VALIDATION_SPLIT,
        callbacks=callbacks,
        verbose=2
    )
    return history

def evaluate_model(model, x_test, y_test):
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test loss: {loss:.4f} - Test accuracy: {acc:.4f}")
    return loss, acc

# -------------------------
# Main: exécution si script
# -------------------------
def main():
    (x_train, y_train), (x_test, y_test), INPUT_SHAPE = prepare_data()

    print("\n--- Entraînement du Basic CNN ---")
    basic_model = build_basic_cnn(INPUT_SHAPE, NUM_CLASSES)
    basic_model.summary()
    compile_and_train(basic_model, x_train, y_train)
    print("\nÉvaluation du Basic CNN sur le jeu de test:")
    evaluate_model(basic_model, x_test, y_test)

    print("\n--- Entraînement du ResNet small ---")
    resnet_model = build_resnet_small(INPUT_SHAPE, NUM_CLASSES)
    resnet_model.summary()
    compile_and_train(resnet_model, x_train, y_train)
    print("\nÉvaluation du ResNet small sur le jeu de test:")
    evaluate_model(resnet_model, x_test, y_test)

if __name__ == "__main__":
    main()


   819200/170498071 ━━━━━━━━━━━━━━━━━━━━ 5:00:08 106us/step